# Neural Network
This script uses a neural network to model the relationship between brains and cognition.

In [3]:
import glob
import os

import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'keras'

## Get Data

## Dense Neural Network

## Convolutional Neural Network